<a href="https://colab.research.google.com/github/Ranjanunicode/transformerBHU/blob/main/poem_gen21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time

import numpy as np
import tensorflow as tf

In [ ]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [ ]:
# text = open("/content/drive/MyDrive/hindi_text/sample_dhoopbahuthai_1.txt", "rb").read().decode(encoding="utf-8")
# print(f"Length of text: {len(text)} characters")

In [ ]:

print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


In [ ]:
example_texts = ["abcdefg", "xyz"]

# TODO 1
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# example_texts = ["राहत", "चहकारें]"]

# # TODO 1
# chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
# chars

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# TODO 2
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([ 5, 43, 32, 30, 19, 49, 42, 61,  8, 52, 29, 24, 23, 53, 60,  9, 60,
       27, 61, 18,  3,  1, 27, 63,  3, 25,  7, 18,  6, 28,  3, 28, 62, 39,
       56, 58, 65, 44,  7, 27,  9, 52, 59, 62, 65, 18,  3, 25, 20, 49, 31,
       26, 36, 14, 30, 52, 53, 14, 49, 10, 38, 56, 27, 58, 37, 21, 23, 41,
       11,  5, 65, 41, 26, 29, 18, 15, 33, 27, 53, 35,  0, 59, 14,  8, 21,
       19, 55,  7, 52, 48, 37, 36, 26,  3, 13, 23, 30, 11, 22, 41])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'm,\nMore than thou hast, and with it joy thy life;\nSo as thou livest in peace, die free from strife:\n'

Next Char Predictions:
 b"&dSQFjcv-mPKJnu.uNvE!\nNx!L,E'O!OwZqsze,N.mtwzE!LGjRMWAQmnAj3YqNsXHJb:&zbMPEBTNnV[UNK]tA-HFp,miXWM!?JQ:Ib"


In [ ]:
# TODO - add a loss function here
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189863, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

66.013756

In [ ]:
model.compile(optimizer="adam", loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

In [ ]:
EPOCHS = 50

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 17s 57ms/step - loss: 2.7525
Epoch 2/50
172/172 [==============================] - 11s 51ms/step - loss: 2.0090
Epoch 3/50
172/172 [==============================] - 11s 55ms/step - loss: 1.7275
Epoch 4/50
172/172 [==============================] - 14s 59ms/step - loss: 1.5629
Epoch 5/50
172/172 [==============================] - 11s 56ms/step - loss: 1.4614
Epoch 6/50
172/172 [==============================] - 12s 53ms/step - loss: 1.3922
Epoch 7/50
172/172 [==============================] - 11s 54ms/step - loss: 1.3390
Epoch 8/50
172/172 [==============================] - 12s 55ms/step - loss: 1.2938
Epoch 9/50
172/172 [==============================] - 14s 59ms/step - loss: 1.2523
Epoch 10/50
172/172 [==============================] - 14s 56ms/step - loss: 1.2144
Epoch 11/50
172/172 [==============================] - 13s 57ms/step - loss: 1.1752
Epoch 12/50
172/172 [==============================] - 12s 56ms/step - loss: 1.1347
E

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
The write is not worse than Judnis,' treasure of again;
Over the devil's man beg root as he,
But with his last lives banish him, so surprised on himself
and the first affection. Play'd and knell
Opportuned he that is low in outring
That they have quenches here: I am palabel, I'll give thee.

KING EDWARD IV:
Seit an our brother, take away for virtuous
Is nothing left are their pettitood and weep
Till I can make my oath in heaven forbid!
Weigh it be so, it shall become of me?
If fortune I have deserved no pity.
The air I love a man to death! a braggart, are you life
On the first knave me of this father.

JULIET:
No, praise me of thy country, and a villain,
Why stand'st thy unhated.
Now, where's my mother bring us to?

PROSPERO:
Thou trill, a happy is to die.

Provost:
He's within; Let me have it musicion.

WARWICK:
O pardon, go, pray thee from thine ornamed
in their ancee thornemning or a rat-up to life:
I pray God Cleapen's easy to be avoided;' bush'd,
And in his piece from heave

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, "\n\n" + "_" * 80)
print("\nRun time:", end - start)

tf.Tensor(
[b"ROMEO:\nThe wish doth Edward here shall take her die.\n\nEXETER:\nHer further than the rest will second came unto;\nAnd say 'What is't your husband and her father is brought:\nThe everless done far from heart since I saw my cousin\nUpon his brother, take it for his death:\nMake haste, or sure, my tender-hearted kept must war\nThurhy tyranny before her fortice;\nFor either wear impeachenties' entertaighments be a kingdom's word,\nThough and burne it, then in men: how cheer his pleasure\nOf fouls of this direful murder o' the maid.\n\nHASTINGS:\nGramers, your sister, then, have you not for me this allowing,\nI tell thee what: get thee in her fellow?\n\nTYRREL:\nPray you, put up; or rather for her keeper was surfeits, it\nwithin me respect.\n\nSICINIUS:\nHe were returned their royal rebels from his grave:\nRind from none so were they from age to age,\nMy vice and trumpets and tread treasury\nOf two sleeping with our comfort.\n\nAUFIDIUS:\nI wish,\nWe may, then know you are b

In [ ]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [ ]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The wifes of hour little babeful womb
Expose these thousand father, or thy mother
Netters to ship a


Advanced: Customized Training The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager execution guide.

The basic procedure is:

Execute the model and calculate the loss under a tf.GradientTape. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {"loss": loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [ ]:
model.fit(dataset, epochs=50)

Epoch 1/50
172/172 [==============================] - 15s 58ms/step - loss: 2.6853
Epoch 2/50
172/172 [==============================] - 13s 57ms/step - loss: 1.9645
Epoch 3/50
172/172 [==============================] - 13s 59ms/step - loss: 1.6886
Epoch 4/50
172/172 [==============================] - 13s 60ms/step - loss: 1.5336
Epoch 5/50
172/172 [==============================] - 12s 58ms/step - loss: 1.4403
Epoch 6/50
172/172 [==============================] - 12s 57ms/step - loss: 1.3727
Epoch 7/50
172/172 [==============================] - 11s 56ms/step - loss: 1.3202
Epoch 8/50
172/172 [==============================] - 12s 57ms/step - loss: 1.2749
Epoch 9/50
172/172 [==============================] - 12s 58ms/step - loss: 1.2319
Epoch 10/50
172/172 [==============================] - 12s 59ms/step - loss: 1.1917
Epoch 11/50
172/172 [==============================] - 12s 59ms/step - loss: 1.1497
Epoch 12/50
172/172 [==============================] - 12s 58ms/step - loss: 1.1068
E

In [ ]:
model.summary()

Model: "custom_training_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save('poem_eng1.h5py')

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for batch_n, (inp, target) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs["loss"])

        if batch_n % 50 == 0:
            template = (
                f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            )
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f"Epoch {epoch+1} Loss: {mean.result().numpy():.4f}")
    print(f"Time taken for 1 epoch {time.time() - start:.2f} sec")
    print("_" * 80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 0.3767
Epoch 1 Batch 50 Loss 0.3980
Epoch 1 Batch 100 Loss 0.4360
Epoch 1 Batch 150 Loss 0.4598

Epoch 1 Loss: 0.4266
Time taken for 1 epoch 14.63 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 0.3742
Epoch 2 Batch 50 Loss 0.3967
Epoch 2 Batch 100 Loss 0.4345
Epoch 2 Batch 150 Loss 0.4603

Epoch 2 Loss: 0.4297
Time taken for 1 epoch 13.25 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 0.3965
Epoch 3 Batch 50 Loss 0.4135
Epoch 3 Batch 100 Loss 0.4288
Epoch 3 Batch 150 Loss 0.4823

Epoch 3 Loss: 0.4269
Time taken for 1 epoch 12.81 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 0.3859
Epoch 4 Batch 50 Loss 0.4079
Epoch 4 Batch 100 Loss 0.4250
Epoch 4 Batch 150 Loss 0.4754

Epoch 4 Loss: 0.4263
Time taken for 1 epoch 11.94 sec
_____________________________________________________________________